# Fermi-LAT via FermiPyLike

In this Example we show how to use the fermipy plugin in threeML. We perform a Binned likelihood analysis and a Bayesian analysis of the Crab, optimizing the parameters of the Crab Pulsar (PSR J0534+2200) keeping fixed the parameters of the Crab Nebula. In the model, the nebula is described by two sources, one representing the synchrotron spectrum, the othet the Inverse Compton emission.
In this example we show how to download Fermi-LAT data, how to build a model starting from the 4FGL, how to free and fix parameters of the sources in the model, and how to perform a spectral analysis using the fermipy plugin.

In [1]:
import warnings

warnings.simplefilter("ignore")
import numpy as np

np.seterr(all="ignore")
import shutil
from IPython.display import Image, display
import glob
from pathlib import Path
import matplotlib as mpl
from matplotlib import pyplot as plt
from astropy.io import fits as pyfits
import scipy as sp

In [2]:
%%capture
from threeML import *

In [3]:
from jupyterthemes import jtplot

%matplotlib inline
jtplot.style(context="talk", fscale=1, ticks=True, grid=False)
set_threeML_style()
silence_warnings()

## The Fermi 4FGL catalog
Let's interrogate the 4FGL to get the sources in a radius of 20.0 deg around the Crab

In [4]:
lat_catalog = FermiLATSourceCatalog()

ra, dec, table = lat_catalog.search_around_source("Crab", radius=20.0)

table

19:02:15 INFO      The cache for fermilpsc does not yet exist. We will try to     ]8;id=573316;file:///Users/runner/work/threeML/threeML/threeML/io/get_heasarc_table_as_pandas.py\get_heasarc_table_as_pandas.py]8;;\:]8;id=474307;file:///Users/runner/work/threeML/threeML/threeML/io/get_heasarc_table_as_pandas.py#64\64]8;;\
                  build it                                                                                         
                                                                                                                   

         INFO      Building cache for fermilpsc                                  ]8;id=779042;file:///Users/runner/work/threeML/threeML/threeML/io/get_heasarc_table_as_pandas.py\get_heasarc_table_as_pandas.py]8;;\:]8;id=798194;file:///Users/runner/work/threeML/threeML/threeML/io/get_heasarc_table_as_pandas.py#112\112]8;;\

Trying https://heasarc.gsfc.nasa.gov/cgi-bin/vo/cone/coneGet.pl?table=fermilpsc&


name,source_type,short_source_type,ra,dec,assoc_name,tevcat_assoc,Search_Offset
,,,deg,deg,,,
object,str52,object,float64,float64,object,object,float64
4FGL J0534.5+2201s,pulsar wind nebula,PWN,83.6331,22.0199,Crab Nebula,Crab,0.3965
4FGL J0534.5+2201i,pulsar wind nebula,PWN,83.6330,22.0200,Crab Nebula,Crab,0.3979
4FGL J0534.5+2200,"pulsar, identified by pulsations",PSR,83.6367,22.0149,PSR J0534+2200,Crab pulsar,0.4447
4FGL J0526.3+2246,active galaxy of uncertain type,bcu,81.5908,22.7778,NVSS J052622+224801,,121.9694
4FGL J0544.4+2238,unknown,,86.1093,22.6418,,,142.7227
4FGL J0521.7+2112,BL Lac type of blazar,bll,80.4445,21.2131,TXS 0518+211,VER J0521+211,184.0105
4FGL J0528.3+1817,unknown,unk,82.0946,18.2943,1RXS J052829.6+181657,,239.3570
4FGL J0519.7+1939,unknown,,79.9457,19.6646,,,250.0485


This gets a 3ML model (a Model instance) from the table above, where every source in the 4FGL becomes a Source instance. Note that by default all parameters of all sources are fixed.

In [5]:
model = lat_catalog.get_model()

Let's free all the normalizations within 3 deg from the center.

In [6]:
model.free_point_sources_within_radius(3.0, normalization_only=True)

model.display()

Model summary:
==============

                    N
Point sources     196
Extended sources    0
Particle sources    0

Free parameters (5):
--------------------

                                        value min_value max_value  \
Crab_synch.spectrum.main.Log_parabola.K   0.0       0.0       0.0   
Crab_IC.spectrum.main.Log_parabola.K      0.0       0.0       0.0   
PSR_J0534p2200...K                        0.0       0.0       0.0   
NVSS_J052622p224801...K                   0.0       0.0       0.0   
x4FGL_J0544d4p2238...K                    0.0       0.0       0.0   

                                                   unit  
Crab_synch.spectrum.main.Log_parabola.K  keV-1 s-1 cm-2  
Crab_IC.spectrum.main.Log_parabola.K     keV-1 s-1 cm-2  
PSR_J0534p2200...K                       keV-1 s-1 cm-2  
NVSS_J052622p224801...K                  keV-1 s-1 cm-2  
x4FGL_J0544d4p2238...K                   keV-1 s-1 cm-2  

Fixed parameters (1089):
(abridged. Use complete=True to see all fixed parameters)


Properties (0):
--------------------

(none)


Linked parameters (0):
----------------------

(none)

Independent variables:
----------------------

(none)

Linked functions (0):
----------------------

(none)

but then let's fix the sync and the IC components of the Crab nebula (cannot fit them with just one month of data) (these two methods are equivalent)

In [7]:
model["Crab_IC.spectrum.main.Log_parabola.K"].fix = True
model.Crab_synch.spectrum.main.Log_parabola.K.fix = True

However, let's free the index of the Crab Pulsar

In [8]:
model.PSR_J0534p2200.spectrum.main.Super_cutoff_powerlaw.index.free = True

model.display()

Model summary:
==============

                    N
Point sources     196
Extended sources    0
Particle sources    0

Free parameters (4):
--------------------

                            value min_value max_value            unit
PSR_J0534p2200...K            0.0       0.0       0.0  keV-1 s-1 cm-2
PSR_J0534p2200...index  -1.826553     -10.0      10.0                
NVSS_J052622p224801...K       0.0       0.0       0.0  keV-1 s-1 cm-2
x4FGL_J0544d4p2238...K        0.0       0.0       0.0  keV-1 s-1 cm-2

Fixed parameters (1090):
(abridged. Use complete=True to see all fixed parameters)


Properties (0):
--------------------

(none)


Linked parameters (0):
----------------------

(none)

Independent variables:
----------------------

(none)

Linked functions (0):
----------------------

(none)

In [9]:
# Download data from Jan 01 2010 to February 1 2010

tstart = "2010-01-01 00:00:00"
tstop = "2010-02-01 00:00:00"

# Note that this will understand if you already download these files, and will
# not do it twice unless you change your selection or the outdir

evfile, scfile = download_LAT_data(
    ra,
    dec,
    20.0,
    tstart,
    tstop,
    time_type="Gregorian",
    destination_directory="Crab_data",
)

19:02:29 INFO      Query parameters:                                                       ]8;id=178141;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=725314;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#262\262]8;;\

         INFO                          coordfield = 83.6287,22.0147                        ]8;id=184936;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=727012;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#266\266]8;;\

         INFO                         coordsystem = J2000                                  ]8;id=621162;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=55529;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#266\266]8;;\

         INFO                          shapefield = 20.0                                   ]8;id=63282;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=220344;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#266\266]8;;\

         INFO                           timefield = 2010-01-01 00:00:00,2010-02-01         ]8;id=762604;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=747146;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#266\266]8;;\
                  00:00:00                                                                                         

...

         INFO                          spacecraft = checked                                ]8;id=724692;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=454849;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#266\266]8;;\

         INFO      Query ID: b7ba925258c89244b49cc1889e66d453                              ]8;id=153935;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=180292;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#271\271]8;;\

19:02:33 INFO      Estimated complete time for your query: 13 seconds                      ]8;id=766791;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=719355;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#428\428]8;;\

         INFO      If this download fails, you can find your data at                       ]8;id=29237;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=378547;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#437\437]8;;\
                  https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/QueryResults.cgi?id=L2402031                         
                  402324596CEF229 (when ready)                                                                     

19:02:50 INFO      Downloading FT1 and FT2 files...                                        ]8;id=394003;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py\download_LAT_data.py]8;;\:]8;id=462526;file:///Users/runner/work/threeML/threeML/threeML/utils/data_download/Fermi_LAT/download_LAT_data.py#527\527]8;;\

## Configuration for Fermipy

3ML provides and intreface into [Fermipy](https://fermipy.readthedocs.io/en/latest/) via the **FermipyLike** plugin. We can use it to generate basic configuration files.


.. note::
    Currently, the FermipyLike plugin does not provide an interface to handle extended sources. This will change


In [10]:
config = FermipyLike.get_basic_config(
    evfile=evfile,
    scfile=scfile,
    ra=ra,
    dec=dec,
    fermipy_verbosity=1,
    fermitools_chatter=0,
)

# See what we just got

config.display()

binning:
  binsperdec: 8
  binsz: 0.1
  roiwidth: 10.0
data:
  evfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/Lb7ba925258c89244b49cc1889e66d453_FT1.fits
  scfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/L2402031402324596CEF229_SC00.fits
logging:
  chatter: 0
  verbosity: 1
selection:
  dec: 22.014694786873797
  emax: 100000.0
  emin: 100.0
  evclass: 128
  evtype: 3
  filter: DATA_QUAL>0 && LAT_CONFIG==1
  ra: 83.62870762486367
  tmax: 286675202.0
  tmin: 283996802.0
  zmax: 100.0



binning:
 binsperdec: 8
 binsz: 0.1
 roiwidth: 10.0
data:
 evfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/Lb7ba925258c89244b49cc1889e66d453_FT1.fits
 scfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/L2402031402324596CEF229_SC00.fits
logging:
 chatter: 0
 verbosity: 1
selection:
 dec: 22.014694786873797
 emax: 100000.0
 emin: 100.0
 evclass: 128
 evtype: 3
 filter: DATA_QUAL>0 && LAT_CONFIG==1
 ra: 83.62870762486367
 tmax: 286675202.0
 tmin: 283996802.0
 zmax: 100.0

You can of course modify the configuration as a dictionary

In [11]:
config["selection"]["emax"] = 300000.0

and even add sections

In [12]:
config["gtlike"] = {"edisp": False}

config.display()

binning:
  binsperdec: 8
  binsz: 0.1
  roiwidth: 10.0
data:
  evfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/Lb7ba925258c89244b49cc1889e66d453_FT1.fits
  scfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/L2402031402324596CEF229_SC00.fits
gtlike:
  edisp: false
logging:
  chatter: 0
  verbosity: 1
selection:
  dec: 22.014694786873797
  emax: 300000.0
  emin: 100.0
  evclass: 128
  evtype: 3
  filter: DATA_QUAL>0 && LAT_CONFIG==1
  ra: 83.62870762486367
  tmax: 286675202.0
  tmin: 283996802.0
  zmax: 100.0



binning:
 binsperdec: 8
 binsz: 0.1
 roiwidth: 10.0
data:
 evfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/Lb7ba925258c89244b49cc1889e66d453_FT1.fits
 scfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/L2402031402324596CEF229_SC00.fits
gtlike:
 edisp: false
logging:
 chatter: 0
 verbosity: 1
selection:
 dec: 22.014694786873797
 emax: 300000.0
 emin: 100.0
 evclass: 128
 evtype: 3
 filter: DATA_QUAL>0 && LAT_CONFIG==1
 ra: 83.62870762486367
 tmax: 286675202.0
 tmin: 283996802.0
 zmax: 100.0

### FermipyLike
Let's create an instance of the plugin/ Note that here no processing is made, because fermipy still doesn't know about the model you want to use.



In [13]:
LAT = FermipyLike("LAT", config)

The plugin modifies the configuration as needed to get the output files in a unique place, which will stay the same as long as your selection does not change.

In [14]:
config.display()

binning:
  binsperdec: 8
  binsz: 0.1
  roiwidth: 10.0
data:
  evfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/Lb7ba925258c89244b49cc1889e66d453_FT1.fits
  scfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/L2402031402324596CEF229_SC00.fits
fileio:
  outdir: __a0d009e9d7f0df5dcbcb6253686984b6
gtlike:
  edisp: false
logging:
  chatter: 0
  verbosity: 1
selection:
  dec: 22.014694786873797
  emax: 300000.0
  emin: 100.0
  evclass: 128
  evtype: 3
  filter: DATA_QUAL>0 && LAT_CONFIG==1
  ra: 83.62870762486367
  tmax: 286675202.0
  tmin: 283996802.0
  zmax: 100.0



binning:
 binsperdec: 8
 binsz: 0.1
 roiwidth: 10.0
data:
 evfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/Lb7ba925258c89244b49cc1889e66d453_FT1.fits
 scfile: /Users/runner/work/threeML/threeML/docs/md_docs/slow_execute/Crab_data/L2402031402324596CEF229_SC00.fits
fileio:
 outdir: __a0d009e9d7f0df5dcbcb6253686984b6
gtlike:
 edisp: false
logging:
 chatter: 0
 verbosity: 1
selection:
 dec: 22.014694786873797
 emax: 300000.0
 emin: 100.0
 evclass: 128
 evtype: 3
 filter: DATA_QUAL>0 && LAT_CONFIG==1
 ra: 83.62870762486367
 tmax: 286675202.0
 tmin: 283996802.0
 zmax: 100.0

Here is where the fermipy processing happens (the .setup method)

In [15]:
fermipy_output_directory = Path(config["fileio"]["outdir"])
print("Fermipy Output directoty: %s" % fermipy_output_directory)

# This remove the output directory, to start a fresh analysis...

if fermipy_output_directory.exists():
    shutil.rmtree(fermipy_output_directory)

# Here is where the fermipy processing happens (the .setup method)

data = DataList(LAT)

jl = JointLikelihood(model, data)

Fermipy Output directoty: __a0d009e9d7f0df5dcbcb6253686984b6


19:03:10 INFO      Using IRFs P8R3_SOURCE_V3                                                     ]8;id=515996;file:///Users/runner/work/threeML/threeML/threeML/plugins/FermipyLike.py\FermipyLike.py]8;;\:]8;id=244315;file:///Users/runner/work/threeML/threeML/threeML/plugins/FermipyLike.py#126\126]8;;\


Found Galactic template for IRF. P8R3_SOURCE_V3: /usr/local/miniconda/envs/test_env/share/fermitools/refdata/fermi/galdiffuse/gll_iem_v07.fits

Cutting the template around the ROI: 




Found Isotropic template for irf P8R3_SOURCE_V3: /usr/local/miniconda/envs/test_env/share/fermitools/refdata/fermi/galdiffuse/iso_P8R3_SOURCE_V3_v1.txt


19:13:47 INFO      set the minimizer to minuit                                             ]8;id=369128;file:///Users/runner/work/threeML/threeML/threeML/classicMLE/joint_likelihood.py\joint_likelihood.py]8;;\:]8;id=967996;file:///Users/runner/work/threeML/threeML/threeML/classicMLE/joint_likelihood.py#1045\1045]8;;\

The normalization factors of the LAT background components are included in the models as nuisance parameters. They are only added during the previous step (during the model assignment). Let's display them:

In [16]:
for k, v in LAT.nuisance_parameters.items():
    print(k, ":", v)

LAT_isodiff_Normalization : Parameter LAT_isodiff_Normalization = 1.0 []
(min_value = 0.001, max_value = 1000.0, delta = 0.01, free = True)
LAT_galdiff_Prefactor : Parameter LAT_galdiff_Prefactor = 1.0 []
(min_value = 0.1, max_value = 10.0, delta = 0.01, free = True)


We will fix the isotropic BG as we are not sensitive to it with this dataset. We will also fix one more weak source.

In [17]:
model.LAT_isodiff_Normalization.fix = True
model.x4FGL_J0544d4p2238.spectrum.main.Powerlaw.K.fix = True
model.display()

Model summary:
==============

                    N
Point sources     196
Extended sources    0
Particle sources    0

Free parameters (4):
--------------------

                            value min_value max_value            unit
PSR_J0534p2200...K            0.0       0.0       0.0  keV-1 s-1 cm-2
PSR_J0534p2200...index  -1.826553     -10.0      10.0                
NVSS_J052622p224801...K       0.0       0.0       0.0  keV-1 s-1 cm-2
LAT_galdiff_Prefactor         1.0       0.1      10.0                

Fixed parameters (1092):
(abridged. Use complete=True to see all fixed parameters)


Properties (0):
--------------------

(none)


Linked parameters (0):
----------------------

(none)

Independent variables:
----------------------

(none)

Linked functions (0):
----------------------

(none)

### Performing the fit

In [18]:
jl.set_minimizer("minuit")

res = jl.fit()

         INFO      set the minimizer to MINUIT                                             ]8;id=170812;file:///Users/runner/work/threeML/threeML/threeML/classicMLE/joint_likelihood.py\joint_likelihood.py]8;;\:]8;id=595750;file:///Users/runner/work/threeML/threeML/threeML/classicMLE/joint_likelihood.py#1062\1062]8;;\

Best fit values:

,result,unit
parameter,,
PSR_J0534p2200...K,(2.46 -0.08 +0.09) x 10^-13,1 / (cm2 keV s)
PSR_J0534p2200...index,-1.790 +/- 0.025,
NVSS_J052622p224801.spectrum.main.Powerlaw.K,(1.0 -0.5 +1.0) x 10^-16,1 / (cm2 keV s)
LAT_galdiff_Prefactor,(9.35 +/- 0.08) x 10^-1,


Correlation matrix:

1.00,0.73,-0.05,-0.11
0.73,1.00,0.10,0.16
-0.05,0.10,1.00,-0.14
-0.11,0.16,-0.14,1.00


Values of -log(likelihood) at the minimum:

,-log(likelihood)
LAT,118073.606157
total,118073.606157


Values of statistical measures:

,statistical measures
AIC,236155.212456
BIC,236197.382493


Now let's compute the errors on the best fit parameters



In [19]:
res = jl.get_errors()

,result,unit
parameter,,
PSR_J0534p2200...K,(2.46 +/- 0.09) x 10^-13,1 / (cm2 keV s)
PSR_J0534p2200...index,-1.790 -0.025 +0.024,
NVSS_J052622p224801.spectrum.main.Powerlaw.K,(10 -6 +7) x 10^-17,1 / (cm2 keV s)
LAT_galdiff_Prefactor,(9.35 -0.17 +0.18) x 10^-1,


We might also want to look at the profile of the likelihood for each parameter.

In [20]:
res = jl.get_contours(
    model.PSR_J0534p2200.spectrum.main.Super_cutoff_powerlaw.index, -2.0, -1.6, 30
)

Profiling likelihood:   0%|          | 0/30 [00:00<?, ?it/s]

In [21]:
res[-1]

Or we might want to produce a contour plot

In [22]:
res = jl.get_contours(
    "PSR_J0534p2200.spectrum.main.Super_cutoff_powerlaw.K",
    2.1e-13,
    2.7e-13,
    20,
    "PSR_J0534p2200.spectrum.main.Super_cutoff_powerlaw.index",
    -2.0,
    -1.7,
    20,
)

Profiling likelihood:   0%|          | 0/400 [00:00<?, ?it/s]

In [23]:
res[-1]

**Pro-trick:** We can also axcess the GTAnalysis object of fermipy:

In [24]:
# res = jl.fit()
# LAT.gta.write_roi('test',make_plots=True)

All the plots are saved in the output directory as png files:



In [25]:
# pngs=Path(f"{fermipy_output_directory}").glob("*png")
# for png in pngs:
#    print(png)
#    my_image=Image(str(png))
#    display(my_image)

We can also plot the resulting model:

In [26]:
energies = sp.logspace(1, 6, 100) * u.MeV
fig, ax = plt.subplots()
# we only want to visualize the relevant sources...
src_to_plot = ["Crab", "PSR_J0534p2200"]
# Now loop over all point sources and plot them
for source_name, point_source in model.point_sources.items():
    for src in src_to_plot:
        if src in source_name:
            # Plot the sum of all components for this source

            ax.loglog(energies, point_source(energies), label=source_name)
            # If there is more than one component, plot them also separately

            if len(point_source.components) > 1:
                for component_name, component in point_source.components.items():
                    ax.loglog(
                        energies,
                        component.shape(energies),
                        "--",
                        label=f"{component_name} of {source_name}",
                    )

# Add a legend
ax.legend(loc=0, frameon=False)

ax.set_xlabel("Energy (MeV)")
ax.set_ylabel(r"Flux (ph cm$^{-2}$ s$^{-1}$ keV$^{-1}$")
ax.set_ylim([1e-18, 1e-8])

# show the plot
fig

We can also do a bayesian analysis.

This will set priors based on the current defined min-max (log-uniform or uniform).

In [27]:
for param in model.free_parameters.values():
    if param.has_transformation():
        param.set_uninformative_prior(Log_uniform_prior)
    else:
        param.set_uninformative_prior(Uniform_prior)

In [28]:
# It's better to remove the output directory,...
shutil.rmtree(fermipy_output_directory)

bayes = BayesianAnalysis(model, data)

19:46:05 INFO      Using IRFs P8R3_SOURCE_V3                                                     ]8;id=379554;file:///Users/runner/work/threeML/threeML/threeML/plugins/FermipyLike.py\FermipyLike.py]8;;\:]8;id=83838;file:///Users/runner/work/threeML/threeML/threeML/plugins/FermipyLike.py#126\126]8;;\


Found Galactic template for IRF. P8R3_SOURCE_V3: /usr/local/miniconda/envs/test_env/share/fermitools/refdata/fermi/galdiffuse/gll_iem_v07.fits

Cutting the template around the ROI: 




Found Isotropic template for irf P8R3_SOURCE_V3: /usr/local/miniconda/envs/test_env/share/fermitools/refdata/fermi/galdiffuse/iso_P8R3_SOURCE_V3_v1.txt


Take care of the nuisance parameters `LAT_isodiff_Normalization` and `LAT_galdiff_Prefactor`, which are only created during the previous step.

In [29]:
model.LAT_isodiff_Normalization.fix = True
model.LAT_galdiff_Prefactor.set_uninformative_prior(Log_uniform_prior)

In [30]:
bayes.set_sampler("emcee")

n_walkers = 10
burn_in = 10
n_samples = 500

bayes.sampler.setup(n_iterations=n_samples, n_burn_in=burn_in, n_walkers=n_walkers)

res = bayes.sample()

19:56:34 INFO      sampler set to emcee                                                    ]8;id=953147;file:///Users/runner/work/threeML/threeML/threeML/bayesian/bayesian_analysis.py\bayesian_analysis.py]8;;\:]8;id=603994;file:///Users/runner/work/threeML/threeML/threeML/bayesian/bayesian_analysis.py#202\202]8;;\

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

20:01:44 INFO      Mean acceptance fraction: 0.4736000000000001                                ]8;id=371773;file:///Users/runner/work/threeML/threeML/threeML/bayesian/emcee_sampler.py\emcee_sampler.py]8;;\:]8;id=960020;file:///Users/runner/work/threeML/threeML/threeML/bayesian/emcee_sampler.py#157\157]8;;\

20:01:45 INFO      fit restored to maximum of posterior                                         ]8;id=542515;file:///Users/runner/work/threeML/threeML/threeML/bayesian/sampler_base.py\sampler_base.py]8;;\:]8;id=727312;file:///Users/runner/work/threeML/threeML/threeML/bayesian/sampler_base.py#178\178]8;;\

...

,result,unit
parameter,,
PSR_J0534p2200...K,(2.526 -0.14 +0.031) x 10^-13,1 / (cm2 keV s)
PSR_J0534p2200...index,-1.7680 -0.05 +0.0030,
NVSS_J052622p224801.spectrum.main.Powerlaw.K,(0.000 +0.012 +1.2) x 10^-16,1 / (cm2 keV s)
LAT_galdiff_Prefactor,(9.40 -0.27 +0.11) x 10^-1,


Values of -log(posterior) at the minimum:

,-log(posterior)
LAT,-118001.117639
total,-118001.117639


Values of statistical measures:

,statistical measures
AIC,236010.235421
BIC,236052.405458
DIC,236018.582696
PDIC,1.936915


You can access to the parameter range like this (HPD):

In [31]:
this_K = bayes.results.get_variates(
    "PSR_J0534p2200.spectrum.main.Super_cutoff_powerlaw.K"
)
this_idx = bayes.results.get_variates(
    "PSR_J0534p2200.spectrum.main.Super_cutoff_powerlaw.index"
)

print("Highest_posterior_density_intervals :")
print(
    "K (68%%):     %10.2e,%10.2e" % this_K.highest_posterior_density_interval(cl=0.68)
)
print(
    "K (95%%):     %10.2e,%10.2e" % this_K.highest_posterior_density_interval(cl=0.95)
)
print(
    "Index (68%%): %10.2e,%10.2e" % this_idx.highest_posterior_density_interval(cl=0.68)
)
print(
    "Index (95%%): %10.2e,%10.2e" % this_idx.highest_posterior_density_interval(cl=0.95)
)

Highest_posterior_density_intervals :
K (68%):       2.40e-13,  2.57e-13
K (95%):       2.28e-13,  2.61e-13
Index (68%):  -1.82e+00, -1.76e+00
Index (95%):  -1.84e+00, -1.74e+00


In [32]:
corner_figure = bayes.results.corner_plot()
corner_figure